## Working with PDFs with images

PDFs contain more than rich formatting - they have images!

<img src="data/imgs/hai_ai-index-report-2025_chapter2_excerpts_1_of_8.jpg" width="250px" />
<img src="data/imgs/hai_ai-index-report-2025_chapter2_excerpts_4_of_8.jpg" width="250px" />
<img src="data/imgs/hai_ai-index-report-2025_chapter2_excerpts_5_of_8.jpg" width="250px" />

How do we work with these for RAG?

### Approach 1 - Extract text and images separately

Some libraries (like `docling`) can extract text and images from PDFs, and convert them into Markdown files.

In [1]:
from pathlib import Path

data_folder = Path("data/pdfs")
output_dir = Path("data/parsed")
output_dir.mkdir(parents=True, exist_ok=True)

In [2]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.types.doc import ImageRefMode

IMAGE_RESOLUTION_SCALE = 2.0


def parse_pdf_with_images(input_doc_path: Path, output_dir: Path):
    # Reference: https://docling-project.github.io/docling/examples/export_figures/
    md_filename = output_dir / f"{input_doc_path.name.split('.')[0]}-parsed-w-imgs.md"
    if md_filename.exists():
        print(f"Skipping {md_filename} as it already exists.")
        return

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    conv_res = doc_converter.convert(input_doc_path)

    # Save markdown with embedded pictures
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)


pdf_names = [f.name for f in data_folder.glob("hai*.pdf") if f.is_file()]

for pdf_fname in pdf_names:
    print(f"Processing file: {pdf_fname}")

    input_doc_path = data_folder / pdf_fname

    print(f"Converting document {input_doc_path} to multimodal pages...")
    parse_pdf_with_images(input_doc_path, output_dir)


Processing file: hai_ai-index-report-2025_chapter2_excerpts.pdf
Converting document data/pdfs/hai_ai-index-report-2025_chapter2_excerpts.pdf to multimodal pages...
Skipping data/parsed/hai_ai-index-report-2025_chapter2_excerpts-parsed-w-imgs.md as it already exists.


In [3]:
md_filepath = Path("data/parsed/hai_ai-index-report-2025_chapter2_excerpts-parsed-w-imgs.md")
md_txt = md_filepath.read_text()
print(md_txt[:1000])

## Chapter 2: Technical Performance

## RAG: Retrieval Augment Generation (RAG)

An  increasingly  common  capability  being  tested  in  LLMs is retrieval-augmented generation (RAG). This approach integrates LLMs  with retrieval mechanisms  to enhance their  response  generation. The  model fi rst  retrieves  relevant information  from fi les  or  documents  and  then  generates  a response tailored to the user's query based on the retrieved content.  RAG  has  diverse  use  cases,  including  answering precise questions from large databases and addressing customer queries using information from company documents.

models. 2024 also saw the release of numerous benchmarks for  evaluating  RAG  systems,  including  Ragnarok  (a  RAG arena battleground) and CRAG (Comprehensive RAG benchmark). Additionally, specialized RAG benchmarks, such as FinanceBench for fi nancial question answering, have been developed to address speci fi c use cases.

## Berkeley Function Calling Leaderboard

In r

#### Chunking text files with images

More complex than just text, since we need to handle images as well.

- Must include entire image string in the chunk
- When vectorizing, optionally include base64 of image
    - Your embedding model must be multimodal

Chunking becomes more complex.

One method: try a specialized library like `chonkie` to handle this

Chonkie offers a variety of chunking strategies:

<img src="images/chonkie_methods.png" />

There isn't going to be a "one size fits all" solution for chunking PDFs with images. But these libraries can help you get started.

Let's try a couple of different approaches:

In [4]:
from chonkie import RecursiveChunker

# Initialize the recursive chunker to chunk Markdown
chunker = RecursiveChunker.from_recipe("markdown", lang="en")

In [5]:
chunk_texts = chunker.chunk(md_txt)

In [6]:
import textwrap

for chunk in chunk_texts[:5]:
    print(f"\n" + "=" * 40)
    print(f"Token count: {chunk.token_count}")
    print(f"Chunk text:")
    wrapped_text = textwrap.fill(chunk.text[:500]+"...", width=80)
    print(textwrap.indent(wrapped_text, "    "))


Token count: 1681
Chunk text:
    ## Chapter 2: Technical Performance  ## RAG: Retrieval Augment Generation (RAG)
    An  increasingly  common  capability  being  tested  in  LLMs is retrieval-
    augmented generation (RAG). This approach integrates LLMs  with retrieval
    mechanisms  to enhance their  response  generation. The  model fi rst  retrieves
    relevant information  from fi les  or  documents  and  then  generates  a
    response tailored to the user's query based on the retrieved content.  RAG  has
    diverse  use  cases,  including  ...

Token count: 1793
Chunk text:
    ## Berkeley Function-Calling Leaderboard Evaluation Data Composition
    ![Image](data/parsed/hai_ai-index-report-2025_chapter2_excerpts-parsed-w-imgs_ar
    tifacts/image_000000_7d6b2eec7e2a34253b28c23b7501ca8fbb4a6b25b9f0adb845f99f865bb
    f518a.png)  Figure 2.2.17 9  | Javascript (AST)          | Chatting Capability
    | |---------------------------|---------------------------------| | 2.5%
    |

Let's try a "semantic" chunker:

In [7]:
from chonkie import SemanticChunker

# Basic initialization with default parameters
chunker = SemanticChunker(
    embedding_model="minishlab/potion-base-8M",  # Default model
    threshold=0.5,                               # Similarity threshold (0-1) or (1-100) or "auto"
    chunk_size=2048,                              # Maximum tokens per chunk
    min_sentences=1                              # Initial sentences per chunk
)

In [8]:
# Chunk text into `chunk_texts` as we've done before
# BEGIN_SOLUTION
chunk_texts = chunker.chunk(md_txt)
# END_SOLUTION

In [9]:
for chunk in chunk_texts[:5]:
    print(f"\n" + "=" * 40)
    print(f"Token count: {chunk.token_count}")
    print(f"Chunk text:")
    wrapped_text = textwrap.fill(chunk.text[:500]+"...", width=80)
    print(textwrap.indent(wrapped_text, "    "))


Token count: 441
Chunk text:
    ## Chapter 2: Technical Performance  ## RAG: Retrieval Augment Generation (RAG)
    An  increasingly  common  capability  being  tested  in  LLMs is retrieval-
    augmented generation (RAG). This approach integrates LLMs  with retrieval
    mechanisms  to enhance their  response  generation. The  model fi rst  retrieves
    relevant information  from fi les  or  documents  and  then  generates  a
    response tailored to the user's query based on the retrieved content.  RAG  has
    diverse  use  cases,  including  ...

Token count: 778
Chunk text:
    |---------------------------|---------------------------------| | 2.5%
    | 10.0%                           | | SQL (AST) 5.0%            | Simple (Exec)
    | | Java (AST)                | 5.0% Multiple (Exec)            | | 5.0% REST
    (Exec)          | 2.5% Parallel (Exec)            | | Relevance 12.0%
    | 2.5% Parallel & Multiple (Exec) | | Parallel & Multiple (AST) | Simple (AST)
    20.0%   

### Set up Weaviate Collection

In [10]:
import weaviate
import os

client = weaviate.connect_to_embedded(
    version="1.32.0",
    headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
    },
    environment_variables={"LOG_LEVEL": "error"}  # Reduce amount of logs
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
client.collections.delete("Chunks")

In [16]:
from weaviate.classes.config import Property, DataType, Configure, Tokenization

client.collections.create(
    name="Chunks",
    properties=[
        Property(
            name="document_title",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk_number",
            data_type=DataType.INT,
        ),
        Property(
            name="filename",
            data_type=DataType.TEXT,
            tokenization=Tokenization.FIELD
        ),
    ],
    vector_config=[
        Configure.Vectors.text2vec_cohere(
            name="default",
            source_properties=["document_title", "chunk"],
            model="embed-v4.0"
        )
    ]
)

In [17]:
chunks = client.collections.use("Chunks")

### Import data

In [18]:
from tqdm import tqdm

with chunks.batch.fixed_size(batch_size=100) as batch:
    for i, chunk_text in tqdm(enumerate(chunk_texts)):
        obj = {
            "document_title": "HAI AI Index Report 2025",
            "filename": "data/pdfs/hai_ai-index-report-2025_chapter2_excerpts.pdf",
            "chunk": chunk_text.text,
            "chunk_number": i + 1,
        }

        # Add object to batch for import with (batch.add_object())
        # BEGIN_SOLUTION
        batch.add_object(
            properties=obj
        )
        # END_SOLUTION

19it [00:00, 8363.08it/s]


### RAG queries

How do we perform RAG in this scenario? 

This is a bit different, because we haven't embedded the images (or stored them in Weaviate).

In this scenario, let's:

- Retrieve text chunks
- Get images referred to in the text
- Convert the images to base64
- Send (retrieved text + images + prompt) to LLM for RAG

In [20]:
response = chunks.query.hybrid(
    query="Latest in self-driving cars",
    limit=10
)

for o in response.objects:
    print(f"\n" + "=" * 40)
    print(o.properties["chunk"][:1000] + "...")



China's  self-driving  revolution  is  also  accelerating,  led  by companies like Baidu's Apollo Go, which reported 988,000 rides across China in Q3 2024, re fl ecting a 20% year-over-year increase. In October 2024, the company was operating 400 robotaxis and announced plans to expand its fl eet to 1,000 by the end of 2025. Pony.AI, another Chinese autonomous vehicle manufacturer, has pledged to scale its robotaxi fl eet from 200 to at least 1,000 vehicles-with expectations that the fl eet will reach 2,000 to 3,000 by the end of 2026. China is leading the way in autonomous vehicle testing, with reports indicating  that  it  is  testing  more  driverless  cars  than  any other country and currently rolling them out across 16 cities. Robotaxis  in  China  are  notably  a ff ordable-even  cheaper, in  some  cases,  than  rides  provided  by  human  drivers.  To support  this growth, China  has prioritized establishing national regulations to govern the deployment of driverless cars.  B

In [21]:
import re

def extract_image_paths(text):
    """Extract image paths from markdown-style image references."""
    pattern = r'!\[.*?\]\((.*?)\)'
    return re.findall(pattern, text)

In [22]:
def get_image_base64s(image_paths, base_path=None):
    import base64
    base64_images = []
    for img_path in image_paths:
        full_path = Path(base_path) / img_path if base_path else Path(img_path)
        image_bytes = full_path.read_bytes()
        base64_string = base64.b64encode(image_bytes).decode("utf-8")
        base64_images.append(base64_string)

    return base64_images

In [23]:
all_chunks = ""
all_images = []

for o in response.objects:
    chunk_text = o.properties["chunk"]
    image_paths = extract_image_paths(chunk_text)
    all_images.extend(get_image_base64s(image_paths, base_path="data/parsed"))

    all_chunks += "\n\n" + chunk_text

In [26]:
task_text = """
What does this tell us about the latest in self-driving cars

Describe the details from the figures as well, if necessary.
""" + "\n\n" + all_chunks

message = {
    "role": "user", "content": [
        {"type": "text", "text": task_text}
    ]
}

for img in all_images:
    content = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": "image/png",
            "data": img,
        }
    }
    # Append `content`` to message["content"]
    # BEGIN_SOLUTION
    message["content"].append(content)
    # END_SOLUTION

In [27]:
import anthropic

anthropic_response = anthropic.Anthropic().messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    # Add [message] as the messages to pass to Claude
    # BEGIN_SOLUTION
    messages=[message]
    # END_SOLUTION
)

In [28]:
print(anthropic_response.content[0].text)

Based on this comprehensive report on self-driving cars, here are the key developments in autonomous vehicle technology:

## Global Deployment Progress

**United States:**
- **Waymo** leads commercially, operating in 4 major cities (Phoenix, San Francisco, Los Angeles, Austin)
- Provides 150,000 paid rides per week, covering over 1 million miles monthly
- Plans to expand testing to 10 additional cities, including challenging weather conditions (snowy regions)
- **Cruise** faced setbacks with license suspension in 2023 due to safety incidents

**China:**
- Experiencing rapid growth with companies like **Baidu's Apollo Go** reporting 988,000 rides in Q3 2024 (20% year-over-year increase)
- **Pony.AI** scaling from 200 to 1,000+ vehicles, targeting 2,000-3,000 by 2026
- Testing across 16 cities - more than any other country
- Robotaxis are notably affordable, sometimes cheaper than human drivers

## Major Technical Innovations

**New Vehicles:**
- **Tesla Cybercab**: Two-passenger autonom

In [29]:
client.close()

{"build_git_commit":"7cebee0421","build_go_version":"go1.24.5","build_image_tag":"HEAD","build_wv_version":"1.32.0","error":"context canceled","level":"error","msg":"replication engine failed to start after FSM caught up","time":"2025-08-26T17:48:43+01:00"}
{"build_git_commit":"7cebee0421","build_go_version":"go1.24.5","build_image_tag":"HEAD","build_wv_version":"1.32.0","error":"cannot find peer","level":"error","msg":"transferring leadership","time":"2025-08-26T17:48:43+01:00"}
